In [ ]:
pip install --quiet --upgrade tensorflow_federated

     |████████████████████████████████| 430kB 2.7MB/s 
     |████████████████████████████████| 296kB 41.5MB/s 
     |████████████████████████████████| 2.8MB 40.5MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 2.2MB 33.2MB/s 
     |████████████████████████████████| 20.0MB 69.6MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 450kB 52.7MB/s 
     |████████████████████████████████| 3.9MB 41.8MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
#pip install --upgrade tensorflow_federated

In [ ]:
#pip install --quiet tensorflow==2.1

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

# TODO(b/148678573,b/148685415): must use the ReferenceExecutor because it
# supports unbounded references and tff.sequence_* intrinsics.
tff.framework.set_default_executor(tff.framework.ReferenceExecutor())

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

'Hello, World!'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

all_buildings = pd.read_csv('/content/drive/My Drive/Dataset/building meta data.csv')
zipfile = np.load('/content/drive/My Drive/Dataset/Office_NewYork_1000-3000_6-10_7.npz', allow_pickle=True)

Building_data = zipfile['data']
Building_data_norm = zipfile['data_norm']
Building_names = zipfile['name']
List_of_names = pd.Series(Building_names).unique()

NUM_EXAMPLES_PER_USER = 10000
BATCH_SIZE = 500

EC_min = np.min(Building_data[:,-1])
EC_max = np.max(Building_data[:,-1])
EC_mean = np.mean(Building_data[:,-1])

def get_data_for_name(Data, Names, name):
  output_sequence = []
  all_samples = [i for i, n in enumerate(Names) if n == name]

  for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
    batch_samples = all_samples[i:i+BATCH_SIZE]
    output_sequence.append({'x':np.array([Data[i,:-1] for i in batch_samples], dtype=np.float32),
                'y':np.array([Data[i,-1] for i in batch_samples], dtype=np.float32)})
  return output_sequence

def get_label_for_name(Data, Names, name):
  output_sequence = []
  all_samples = [i for i, n in enumerate(Names) if n == name]

  for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
    batch_samples = all_samples[i:i+BATCH_SIZE]
    output_sequence.append({'y':np.array([Data[i,-1] for i in batch_samples], dtype=np.float32)})
  return output_sequence


federated_train_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[:-2]]

federated_valid_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[-2:-1]]

local_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[-1:]]

local_train_data = local_data[0][:2]

local_test_data = local_data[0][-2:]

Label_test_data = [get_label_for_name(Building_data, Building_names, name) for name in List_of_names[-1:]][0][-2:]

In [ ]:
federated_train_data[4][-1]['y']

array([0.20318568, 0.2237399 , 0.2491381 , 0.24848352, 0.24521056,
       0.25214925, 0.25699323, 0.25450578, 0.2609208 , 0.24573424,
       0.2229544 , 0.21640848, 0.22125246, 0.22622737, 0.2264892 ,
       0.22544184, 0.23264237, 0.23198777, 0.23238054, 0.23473707,
       0.23656993, 0.23146412, 0.24442504, 0.24861443, 0.30504036,
       0.28775913, 0.3495527 , 0.34470868, 0.35897884, 0.36041892,
       0.39341044, 0.3432686 , 0.35766965, 0.37665287, 0.29456687,
       0.25345844, 0.23997381, 0.23251146, 0.23552257, 0.23146412,
       0.23355879, 0.23761728, 0.23958106, 0.23997381, 0.24010472,
       0.24953088, 0.24874537, 0.25031638, 0.28998473, 0.3221907 ,
       0.37612915, 0.40820426, 0.381235  , 0.35662228, 0.3648702 ,
       0.37586734, 0.35112372, 0.33083135, 0.30058914, 0.25162557,
       0.24953088, 0.23958106, 0.23604625, 0.23264237, 0.2292385 ,
       0.22635828, 0.2271438 , 0.2293694 , 0.2271438 , 0.22766747,
       0.23473707, 0.24324678, 0.2939123 , 0.33109316, 0.40100

In [ ]:
EC_mean_test = (np.sum(Label_test_data[0]['y'])+np.sum(Label_test_data[1]['y'])) / (len(Label_test_data[0]['y'])+len(Label_test_data[1]['y']))
print(EC_mean_test)
print(EC_mean)

59.98853926651306
32.58286671659921


In [ ]:
BATCH_SPEC = collections.OrderedDict(x=tf.TensorSpec(shape=[None,6], dtype=tf.float32), y=tf.TensorSpec(shape=[None], dtype=tf.float32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

'<x=float32[?,6],y=float32[?]>'

In [ ]:
MODEL_SPEC = collections.OrderedDict(weights1=tf.TensorSpec(shape=[6,5], dtype=tf.float32),
                                     weights2=tf.TensorSpec(shape=[5,5], dtype=tf.float32),
                                     weights3=tf.TensorSpec(shape=[5,1], dtype=tf.float32),
                                     bias1=tf.TensorSpec(shape=[5], dtype=tf.float32),
                                     bias2=tf.TensorSpec(shape=[5], dtype=tf.float32),
                                     bias3=tf.TensorSpec(shape=[1], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>


In [ ]:
MODEL_SPEC = collections.OrderedDict(weights1=tf.TensorSpec(shape=[6,5], dtype=tf.float32),
                                     weights2=tf.TensorSpec(shape=[5,5], dtype=tf.float32),
                                     weights3=tf.TensorSpec(shape=[5,5], dtype=tf.float32),
                                     weights4=tf.TensorSpec(shape=[5,1], dtype=tf.float32),
                                     bias1=tf.TensorSpec(shape=[5], dtype=tf.float32),
                                     bias2=tf.TensorSpec(shape=[5], dtype=tf.float32),
                                     bias3=tf.TensorSpec(shape=[5], dtype=tf.float32),
                                     bias4=tf.TensorSpec(shape=[1], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,5],weights4=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[5],bias4=float32[1]>


In [ ]:
# NOTE: `forward_pass` is defined separately from `batch_loss` so that it can 
# be later called from within another tf.function. Necessary because a
# @tf.function  decorated method cannot invoke a @tff.tf_computation.

@tf.function
def forward_pass(model, batch):
  y1 = tf.nn.relu(tf.matmul(batch['x'],model['weights1']) + model['bias1'])
  y2 = tf.nn.relu(tf.matmul(y1,model['weights2']) + model['bias2'])
  predicted_y = tf.matmul(y2,model['weights3']) + model['bias3']
  return tf.reduce_mean(tf.square(batch['y'] - predicted_y))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
  return forward_pass(model, batch)

In [ ]:
# NOTE: `forward_pass` is defined separately from `batch_loss` so that it can 
# be later called from within another tf.function. Necessary because a
# @tf.function  decorated method cannot invoke a @tff.tf_computation.

@tf.function
def forward_pass(model, batch):
  y1 = tf.nn.relu(tf.matmul(batch['x'],model['weights1']) + model['bias1'])
  y2 = tf.nn.relu(tf.matmul(y1,model['weights2']) + model['bias2'])
  y3 = tf.nn.relu(tf.matmul(y2,model['weights3']) + model['bias3'])
  predicted_y = tf.matmul(y3,model['weights4']) + model['bias4']
  return tf.reduce_mean(tf.square(batch['y'] - predicted_y))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
  return forward_pass(model, batch)

In [ ]:
str(batch_loss.type_signature)

'(<<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>,<x=float32[?,6],y=float32[?]>> -> float32)'

In [ ]:
@tf.function
def forward(model, batch):
  y1 = tf.nn.relu(tf.matmul(batch['x'],model['weights1']) + model['bias1'])
  y2 = tf.nn.relu(tf.matmul(y1,model['weights2']) + model['bias2'])
  predicted_y = tf.matmul(y2,model['weights3']) + model['bias3']
  return predicted_y

In [ ]:
@tf.function
def forward(model, batch):
  y1 = tf.nn.relu(tf.matmul(batch['x'],model['weights1']) + model['bias1'])
  y2 = tf.nn.relu(tf.matmul(y1,model['weights2']) + model['bias2'])
  y3 = tf.nn.relu(tf.matmul(y2,model['weights3']) + model['bias3'])
  predicted_y = tf.matmul(y3,model['weights4']) + model['bias4']
  return predicted_y

In [ ]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def Metrics_CVRMSE(model, batch):
  prediction = forward(model, batch)
  return (tf.sqrt(tf.reduce_mean(tf.square(batch['y'] - prediction))) / tf.reduce_mean(batch['y'])) * (1-EC_min/(tf.reduce_mean(batch['y'])*(EC_max-EC_min)+EC_min))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def Metrics_RMSE(model, batch):
  prediction = forward(model, batch)
  return tf.sqrt(tf.reduce_mean(tf.square(batch['y'] - prediction))) * (EC_max-EC_min)

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def Metrics_MAE(model, batch):
  prediction = forward(model, batch)
  return tf.reduce_mean(tf.abs(batch['y'] - prediction)) * (EC_max-EC_min)

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def Metrics_MAPE(model, batch):
  prediction = forward(model, batch)
  return tf.reduce_mean(tf.abs(batch['y'] - prediction) / (batch['y']+EC_min/(EC_max-EC_min)))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def Metrics_R2(model, batch):
  prediction = forward(model, batch)
  return 1 - tf.reduce_sum(tf.square(batch['y'] - prediction)) / tf.reduce_sum(tf.square(batch['y'] - tf.reduce_mean(batch['y'])))

In [ ]:
initial_model = collections.OrderedDict(weights1=np.zeros(shape=[6,5], dtype=np.float32),
                                     weights2=np.zeros(shape=[5,5], dtype=np.float32),
                                     weights3=np.zeros(shape=[5,1], dtype=np.float32),
                                     bias1=np.zeros(shape=[5], dtype=np.float32),
                                     bias2=np.zeros(shape=[5], dtype=np.float32),
                                     bias3=np.zeros(shape=[1], dtype=np.float32))

sample_batch = federated_train_data[4][-2]

batch_loss(initial_model, sample_batch)

0.0629788

In [ ]:
initial_model = collections.OrderedDict(weights1=np.zeros(shape=[6,5], dtype=np.float32),
                                     weights2=np.zeros(shape=[5,5], dtype=np.float32),
                                     weights3=np.zeros(shape=[5,5], dtype=np.float32),
                                     weights4=np.zeros(shape=[5,1], dtype=np.float32),
                                     bias1=np.zeros(shape=[5], dtype=np.float32),
                                     bias2=np.zeros(shape=[5], dtype=np.float32),
                                     bias3=np.zeros(shape=[5], dtype=np.float32),
                                     bias4=np.zeros(shape=[1], dtype=np.float32))

sample_batch = federated_train_data[4][-2]

batch_loss(initial_model, sample_batch)

In [ ]:
Metrics_CVRMSE(initial_model, sample_batch)

0.9921009

In [ ]:
Metrics_RMSE(initial_model, sample_batch)

47.922092

In [ ]:
Metrics_MAE(initial_model, sample_batch)

46.67863

In [ ]:
Metrics_MAPE(initial_model, sample_batch)

0.9646735

In [ ]:
Metrics_R2(initial_model, sample_batch)

-9760.624

In [ ]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
  # Define a group of model variables and set them to `initial_model`. Must
  # be defined outside the @tf.function.
  model_vars = collections.OrderedDict([(name, tf.Variable(name=name, initial_value=value)) for name, value in initial_model.items()])
  optimizer = tf.keras.optimizers.SGD(learning_rate)

  @tf.function
  def _train_on_batch(model_vars, batch):
    # Perform one step of gradient descent using loss from `batch_loss`.
    with tf.GradientTape() as tape:
      loss = forward_pass(model_vars, batch)
    grads = tape.gradient(loss, model_vars)
    optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
    return model_vars

  return _train_on_batch(model_vars, batch)

In [ ]:
str(batch_train.type_signature)

'(<<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>,<x=float32[?,6],y=float32[?]>,float32> -> <weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>)'

In [ ]:
model = initial_model
losses = []
CVRMSE = []
for _ in range(5):
  model = batch_train(model, sample_batch, 0.1)
  losses.append(batch_loss(model, sample_batch))
  CVRMSE.append(Metrics_CVRMSE(model, sample_batch))

In [ ]:
losses

[0.041467734, 0.02770063, 0.018889714, 0.013250715, 0.009641752]

In [ ]:
CVRMSE

[0.80503327, 0.6579661, 0.5433396, 0.45507017, 0.38818312]

In [ ]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):

  # Mapping function to apply to each batch.
  @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
  def batch_fn(model, batch):
    return batch_train(model, batch, learning_rate)

  return tff.sequence_reduce(all_batches, initial_model, batch_fn)

In [ ]:
str(local_train.type_signature)

'(<<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>,float32,<x=float32[?,6],y=float32[?]>*> -> <weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>)'

In [ ]:
@tff.tf_computation(tf.float32, tf.float32)
def division(a, b):
  return a / b

In [ ]:
@tff.tf_computation(tf.float32, tf.float32)
def division(a, b):
  return tf.divide(a, b)

In [ ]:
@tff.tf_computation()
def get_one():
  return 1.0

In [ ]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
  # TODO(b/120157713): Replace with `tff.sequence_average()` once implemented.
  return tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE), all_batches))

In [ ]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval_CVRMSE(model, all_batches):
  sum_of_sequence = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: Metrics_CVRMSE(model, b), BATCH_TYPE), all_batches))
  num = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: 1.0, BATCH_TYPE), all_batches))
  return division(sum_of_sequence, num)

@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval_RMSE(model, all_batches):
  sum_of_sequence = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: Metrics_RMSE(model, b), BATCH_TYPE), all_batches))
  num = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: 1.0, BATCH_TYPE), all_batches))
  return division(sum_of_sequence, num)

@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval_MAE(model, all_batches):
  sum_of_sequence = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: Metrics_MAE(model, b), BATCH_TYPE), all_batches))
  num = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: 1.0, BATCH_TYPE), all_batches))
  return division(sum_of_sequence, num)

@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval_MAPE(model, all_batches):
  sum_of_sequence = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: Metrics_MAPE(model, b), BATCH_TYPE), all_batches))
  num = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: 1.0, BATCH_TYPE), all_batches))
  return division(sum_of_sequence, num)

@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval_R2(model, all_batches):
  sum_of_sequence = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: Metrics_R2(model, b), BATCH_TYPE), all_batches))
  num = tff.sequence_sum(tff.sequence_map(tff.federated_computation(lambda b: 1.0, BATCH_TYPE), all_batches))
  return division(sum_of_sequence, num)

In [ ]:
str(local_eval_CVRMSE.type_signature)

'(<<weights1=float32[6,5],weights2=float32[5,5],weights3=float32[5,1],bias1=float32[5],bias2=float32[5],bias3=float32[1]>,<x=float32[?,6],y=float32[?]>*> -> float32)'

In [ ]:
locally_trained_model = local_train(initial_model, 0.1, federated_train_data[4])

In [ ]:
print('initial_model loss =', local_eval(initial_model, federated_train_data[4]))
print('locally_trained_model loss =', local_eval(locally_trained_model, federated_train_data[4]))

initial_model loss = 0.49093497
locally_trained_model loss = 0.029914347


In [ ]:
print('initial_model CVRMSE =', local_eval_CVRMSE(initial_model, federated_train_data[4]))
print('locally_trained_model RMSE =', local_eval_RMSE(locally_trained_model, federated_train_data[4]))
print('locally_trained_model CVRMSE =', local_eval_CVRMSE(locally_trained_model, federated_train_data[4]))

initial_model CVRMSE = 0.983731
locally_trained_model RMSE = 11.177309
locally_trained_model CVRMSE = 0.22980188


In [ ]:
print('initial_model loss =', local_eval_CVRMSE(initial_model, federated_train_data[0]))
print('locally_trained_model loss =', local_eval_CVRMSE(locally_trained_model, federated_train_data[0]))

initial_model loss = 0.827339
locally_trained_model loss = 3.9903874


In [ ]:
SERVER_MODEL_TYPE = tff.FederatedType(MODEL_TYPE, tff.SERVER)
CLIENT_DATA_TYPE = tff.FederatedType(LOCAL_DATA_TYPE, tff.CLIENTS)

In [ ]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
  return tff.federated_mean(tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [ ]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval_CVRMSE(model, data):
  return tff.federated_mean(tff.federated_map(local_eval_CVRMSE, [tff.federated_broadcast(model), data]))

@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval_RMSE(model, data):
  return tff.federated_mean(tff.federated_map(local_eval_RMSE, [tff.federated_broadcast(model), data]))

@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval_MAE(model, data):
  return tff.federated_mean(tff.federated_map(local_eval_MAE, [tff.federated_broadcast(model), data]))

@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval_MAPE(model, data):
  return tff.federated_mean(tff.federated_map(local_eval_MAPE, [tff.federated_broadcast(model), data]))

@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval_R2(model, data):
  return tff.federated_mean(tff.federated_map(local_eval_R2, [tff.federated_broadcast(model), data]))

In [ ]:
print('initial_model loss =', federated_eval(initial_model, federated_train_data))
print('locally_trained_model loss =', federated_eval(locally_trained_model, federated_train_data))

initial_model loss = 0.3311762
locally_trained_model loss = 0.20935066


In [ ]:
print('initial_model CVRMSE =', federated_eval_CVRMSE(initial_model, federated_train_data))
print('locally_trained_model CVRMSE =', federated_eval_CVRMSE(locally_trained_model, federated_train_data))

initial_model CVRMSE = 0.96025115
locally_trained_model CVRMSE = 2.2337132


In [ ]:
SERVER_FLOAT_TYPE = tff.FederatedType(tf.float32, tff.SERVER)

@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE, CLIENT_DATA_TYPE)
def federated_train(model, learning_rate, data):
  return tff.federated_mean(tff.federated_map(local_train, [tff.federated_broadcast(model), tff.federated_broadcast(learning_rate), data]))

In [ ]:
initial_model = collections.OrderedDict(weights1=np.random.rand(6,5).astype(np.float32),
                    weights2=np.random.rand(5,5).astype(np.float32),
                    weights3=np.random.rand(5,1).astype(np.float32),
                    bias1=np.random.rand(5).astype(np.float32),
                    bias2=np.random.rand(5).astype(np.float32),
                    bias3=np.random.rand(1).astype(np.float32))

In [ ]:
fed_model = initial_model
learning_rate = 0.1
loss = []
for round_num in range(50):
  fed_model = federated_train(fed_model, learning_rate, federated_train_data)
  learning_rate = learning_rate * 0.9
  loss_ = federated_eval(fed_model, federated_train_data)
  RMSE = federated_eval_RMSE(fed_model, federated_train_data)
  CVRMSE = federated_eval_CVRMSE(fed_model, federated_train_data)
  loss.append(loss_) 
  if len(loss) > 1:
    if ((loss[-2]-loss[-1]) / loss[-2] < 0.0001):
      print('Train process is done')
      break
  print('round {}, loss={}, RMSE={}, CVRMSE={}'.format(round_num, loss_, RMSE, CVRMSE))

round 0, loss=3.5958471298217773, RMSE=125.65668487548828, CVRMSE=7.9549713134765625
round 1, loss=0.3102833330631256, RMSE=27.470705032348633, CVRMSE=0.8431023955345154
round 2, loss=0.18031781911849976, RMSE=22.725921630859375, CVRMSE=1.1795583963394165
round 3, loss=0.1737377643585205, RMSE=23.714635848999023, CVRMSE=1.412951111793518
round 4, loss=0.173425555229187, RMSE=24.03350067138672, CVRMSE=1.4761743545532227
Train process is done


In [ ]:
print('initial_model train CVRMSE =', federated_eval_CVRMSE(initial_model, federated_train_data))
print('trained_model train CVRMSE =', federated_eval_CVRMSE(fed_model, federated_train_data))

initial_model train CVRMSE = 223.89607
trained_model train CVRMSE = 1.493701


In [ ]:
print('initial_model test RMSE =', local_eval_RMSE(initial_model, local_test_data))
print('trained_model test RMSE =', local_eval_RMSE(fed_model, local_test_data))

initial_model test RMSE = 3072.248
trained_model test RMSE = 31.971203


In [ ]:
print('trained_model test CVRMSE =', local_eval_RMSE(fed_model, local_test_data)/EC_mean_test)

trained_model test CVRMSE = 0.5329551817940137


In [ ]:
print('initial_model test CVRMSE =', local_eval_CVRMSE(initial_model, local_test_data))
print('trained_model test CVRMSE =', local_eval_CVRMSE(fed_model, local_test_data))

initial_model test CVRMSE = 50.99831
trained_model test CVRMSE = 0.5306189


In [ ]:
local_model = fed_model
learning_rate = 0.1
loss = []
for round_num in range(50):
  local_model = local_train(local_model, learning_rate, local_train_data)
  learning_rate = learning_rate * 0.9
  loss_ = local_eval(local_model, local_train_data)
  RMSE = local_eval_RMSE(local_model, local_train_data)
  CVRMSE = local_eval_CVRMSE(local_model, local_train_data)
  loss.append(loss_) 
  if len(loss) > 1:
    if ((loss[-2]-loss[-1]) / loss[-2] < 0.0001):
      print('Train process is done')
      break
  print('round {}, loss={}, RMSE={}, CVRMSE={}'.format(round_num, loss_, RMSE, CVRMSE))

round 0, loss=0.023509668186306953, RMSE=20.703439712524414, CVRMSE=0.3473283648490906
round 1, loss=0.011748472228646278, RMSE=14.635294914245605, CVRMSE=0.24552641808986664
round 2, loss=0.006828448735177517, RMSE=11.1571683883667, CVRMSE=0.1871756911277771
round 3, loss=0.004589683376252651, RMSE=9.146533012390137, CVRMSE=0.15344424545764923
round 4, loss=0.0034926123917102814, RMSE=7.9782562255859375, CVRMSE=0.13384458422660828
round 5, loss=0.002918635029345751, RMSE=7.2927751541137695, CVRMSE=0.12234457582235336
round 6, loss=0.0026004172395914793, RMSE=6.883371829986572, CVRMSE=0.11547618359327316
round 7, loss=0.0024146903306245804, RMSE=6.632757663726807, CVRMSE=0.11127173900604248
round 8, loss=0.0023012273013591766, RMSE=6.474887847900391, CVRMSE=0.10862323641777039
round 9, loss=0.002229036297649145, RMSE=6.372414588928223, CVRMSE=0.10690408945083618
round 10, loss=0.002181412884965539, RMSE=6.303908348083496, CVRMSE=0.1057547926902771
round 11, loss=0.0021489672362804413, 

In [ ]:
print('initial_model test RMSE =', local_eval_RMSE(initial_model, local_test_data))
print('local_model test RMSE =', local_eval_RMSE(local_model, local_test_data))

initial_model test RMSE = 3072.248
local_model test RMSE = 5.2883606


In [ ]:
print('local_model test CVRMSE =', local_eval_RMSE(local_model, local_test_data)/EC_mean_test)

local_model test CVRMSE = 0.0881561821702034


In [ ]:
print('initial_model test CVRMSE =', local_eval_CVRMSE(initial_model, local_test_data))
print('local_model test CVRMSE =', local_eval_CVRMSE(local_model, local_test_data))

initial_model test CVRMSE = 50.99831
local_model test CVRMSE = 0.087783575
